# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

### Metrics to consider:
- Monthly active users
- Daily active users in past month
- Total paid and unpaid users
- Total ads served in the past month
- Cohort per Month - % of users cancelled, % of users upgrades


# Import Libraries

In [59]:
# import libraries
import seaborn as sns
import pandas as pd
import numpy as np
import datetime # missing and together with "pip install --upgrade pyspark" were causing issues with time series
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
from plotly.offline import iplot
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")
import cufflinks as cf
cf.go_offline()
from pyspark.ml.feature import RegexTokenizer, VectorAssembler, Normalizer, StandardScaler
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [60]:
# create a Spark session
# spark = SparkSession.builder.getOrCreate()
spark = SparkSession \
    .builder \
    .appName("Exploring Data") \
    .getOrCreate()

In [61]:
spark

In [62]:
print(spark.catalog.listTables())

[]


In [63]:
pwd

'/home/freemo/Projects/SparkProject'

In [64]:
mini = '/home/freemo/Projects/largeData/mini_sparkify_event_data.json'
medium = '/home/freemo/Projects/largeData/medium_sparkify_event_data.json'

In [65]:
df = spark.read.json(mini)

In [66]:
df.take(1)

[Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30')]

In [67]:
df.show(5)

+----------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|          artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+----------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|  Martha Tilston|Logged In|    Colin|     M|           50| Freeman|277.89016| paid|     Bakersfield, CA|   PUT|NextSong|1538173362000|       29|           Rockpools|   200|1538352117000|Mozilla/5.0 (Wind...|    30|
|Five Iron Frenzy|Logged In|    Micah|     M|           79|    Long|236.09424| free|Boston-Cambridge-...|   PUT|NextSong|1538331630000| 

In [68]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [69]:
# check columns with Null values
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+-----+------+---+---------+------+
|artist|auth|firstName|gender|itemInSession|lastName|length|level|location|method|page|registration|sessionId| song|status| ts|userAgent|userId|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+-----+------+---+---------+------+
| 58392|   0|     8346|  8346|            0|    8346| 58392|    0|    8346|     0|   0|        8346|        0|58392|     0|  0|     8346|     0|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+-----+------+---+---------+------+



In [70]:
# missing values in userID
df.select([count(when(isnan('userID'),True))]).show()

+--------------------------------------------+
|count(CASE WHEN isnan(userID) THEN true END)|
+--------------------------------------------+
|                                           0|
+--------------------------------------------+



In [71]:
# missing values in sessionID
df.select([count(when(isnan('sessionID'),True))]).show()

+-----------------------------------------------+
|count(CASE WHEN isnan(sessionID) THEN true END)|
+-----------------------------------------------+
|                                              0|
+-----------------------------------------------+



In [24]:
# get_hour = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0). hour)
# df = df.withColumn("hour", get_hour(df.ts))
# songs_in_hour = df.filter(df.page == "NextSong").groupby(df.hour).count().orderBy(df.hour.cast("float"))
# songs_in_hour.show()

# songs_in_hour_pd = songs_in_hour.toPandas()
# songs_in_hour_pd.hour = pd.to_numeric(songs_in_hour_pd.hour)

In [72]:
# create udf (user defined functions) for workdays and hour columns
day_name= ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday']
day_ind = [1,2,3,4,5,6,7]
get_hour = udf(lambda x: datetime.datetime.fromtimestamp(x/1000).hour, IntegerType() )
get_day = udf(lambda x: datetime.datetime.fromtimestamp(x/1000).day, IntegerType() )
get_wkday = udf(lambda x: day_name[datetime.datetime.fromtimestamp(x/1000).weekday()] )

In [102]:
df.groupBy('hour').count().sort('hour').show(31)

+----+-----+
|hour|count|
+----+-----+
|   0|12853|
|   1|12334|
|   2|11818|
|   3|11086|
|   4|10462|
|   5|10392|
|   6|10318|
|   7|10098|
|   8|10116|
|   9|10160|
|  10|10254|
|  11|10327|
|  12|10725|
|  13|10863|
|  14|11632|
|  15|12025|
|  16|12699|
|  17|13946|
|  18|14376|
|  19|14731|
|  20|14219|
|  21|13867|
|  22|13633|
|  23|13566|
+----+-----+



In [73]:
# create new columns based on ts column for analyses
df = df.withColumn('hour', get_hour('ts'))
df = df.withColumn('day', get_day('ts'))
df = df.withColumn('workday', get_wkday('ts'))

In [107]:
df.select('hour', 'day', 'workday').show(7)

+----+---+-------+
|hour|day|workday|
+----+---+-------+
|   3|  1| Monday|
|   3|  1| Monday|
|   3|  1| Monday|
|   3|  1| Monday|
|   3|  1| Monday|
|   3|  1| Monday|
|   3|  1| Monday|
+----+---+-------+
only showing top 7 rows



In [75]:
# df.groupby('ts_hour').count().show()

In [76]:
# df.groupby('ts_day').count().show(31)

In [81]:
user_log_valid = df.dropna(how = "any", subset = ["userId", "sessionId"])

In [82]:
user_log_valid.count()

286500

In [84]:
df.select("userId").dropDuplicates().sort("userId").show()

+------+
|userId|
+------+
|      |
|    10|
|   100|
|100001|
|100002|
|100003|
|100004|
|100005|
|100006|
|100007|
|100008|
|100009|
|100010|
|100011|
|100012|
|100013|
|100014|
|100015|
|100016|
|100017|
+------+
only showing top 20 rows



In [85]:
user_log_valid = user_log_valid.filter(user_log_valid["userId"] != "")

In [87]:
user_log_valid.count() - 286500

-8346

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

In [90]:
# check the page column
df.select("page").dropDuplicates().sort("page").show()

+--------------------+
|                page|
+--------------------+
|               About|
|          Add Friend|
|     Add to Playlist|
|              Cancel|
|Cancellation Conf...|
|           Downgrade|
|               Error|
|                Help|
|                Home|
|               Login|
|              Logout|
|            NextSong|
|            Register|
|         Roll Advert|
|       Save Settings|
|            Settings|
|    Submit Downgrade|
| Submit Registration|
|      Submit Upgrade|
|         Thumbs Down|
+--------------------+
only showing top 20 rows



In [132]:
# check a random user
df.select(["userId", "firstname", "page", "level", "song"]).where(df.userId == "98").collect(5)

TypeError: collect() takes 1 positional argument but 2 were given

In [110]:
# create a udf to flag Churned customers
flag_downgrade_event = udf(lambda x: 1 if x == "Cancellation Confirmation" else 0, IntegerType())

In [111]:
# create new column "churned"
df = df.withColumn("churned", flag_downgrade_event("page"))

In [112]:
df.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30', hour=3, day=1, workday='Monday', churned=0)

In [115]:
# add a staging: stage=1 if the user is paid, stage=2 if the user churned
windowval = Window.partitionBy("userId").orderBy(desc("ts")).rangeBetween(Window.unboundedPreceding, 0)

In [116]:
# create new column stage to assign values
df = df.withColumn("phase", Fsum("churned").over(windowval))

In [131]:
# check result
df.select(["userId", "firstname", "ts", "page", "level", "phase"]).where(df.userId == "98").sort("ts").collect()

[Row(userId='98', firstname='Sawyer', ts=1538413513000, page='Home', level='free', phase=0),
 Row(userId='98', firstname='Sawyer', ts=1538413514000, page='Add Friend', level='free', phase=0),
 Row(userId='98', firstname='Sawyer', ts=1538413515000, page='Add Friend', level='free', phase=0),
 Row(userId='98', firstname='Sawyer', ts=1538413647000, page='Home', level='free', phase=0),
 Row(userId='98', firstname='Sawyer', ts=1538413879000, page='NextSong', level='free', phase=0),
 Row(userId='98', firstname='Sawyer', ts=1538414052000, page='NextSong', level='free', phase=0),
 Row(userId='98', firstname='Sawyer', ts=1538414271000, page='NextSong', level='free', phase=0),
 Row(userId='98', firstname='Sawyer', ts=1538414272000, page='Thumbs Down', level='free', phase=0),
 Row(userId='98', firstname='Sawyer', ts=1538414479000, page='NextSong', level='free', phase=0),
 Row(userId='98', firstname='Sawyer', ts=1538414480000, page='Thumbs Down', level='free', phase=0),
 Row(userId='98', firstname=

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [133]:
songs = df.groupby('song')

In [134]:
songs.count().show()

+--------------------+-----+
|                song|count|
+--------------------+-----+
|Never Gonna Be Al...|   50|
|         TULENLIEKKI|   16|
|      Underwaterboys|    1|
|Saor (Free)/News ...|    1|
|               New E|    1|
|The Geeks Were Right|    2|
|              Heaven|   86|
|      Call Me Manana|    6|
|             Push It|   32|
|Cool Monsoon (Wea...|    3|
|The Fun Lovin' Cr...|    3|
|Turn Your Lights ...|    8|
|          Growing Up|    6|
|          Positivity|    2|
|         Miracle Man|    2|
|        I'm The Drug|    3|
|I've Just Seen A ...|   10|
|            Anna May|    1|
|All The Things Sh...|    1|
|    Blue Suede Shoes|    3|
+--------------------+-----+
only showing top 20 rows



In [135]:
churned = df.groupBy('churned')

In [ ]:
churned.sum()

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.